### Penerapan Kalkulasi TF-IDF Untuk Dataset Klasifikasi Komentar

#### Inisialisasi Library Yang Diperlukan Untuk TF-IDF

In [211]:
import math

#### Kalkulasi TF-IDF

In [212]:
# Inisialisasi Class yang akan digunakan untuk kalkulasi TF-IDF
class TFIDFVectorizer:
    # Inisialisasi variable yang dibutuhkan untuk kalkulasi TF-IDF
    def __init__(self) -> None:
        # Inisialisasi variable untuk menampung array teks yang sudah ditokenisasi dalam bentuk dictionary.
        self.documents = {}

        # Inisialisasi variable untuk menampung array TF yang sudah dikalkulasi dengan format {index : {term : value, ...}}
        self.tf = {}

        # Inisialisasi variable untuk menampung array IDF yang sudah dikalkulasi dengan format {key : value}
        self.idf = {}
        self.idf_token = []

        # Inisialisasi variable untuk menampung array TF-IDF yang sudah dikalkulasi dengan format {index : {term : value, ...}}
        self.tfidf = {}

        # Inisialisasi variable untuk menampung hasil tertinggi dan terendah dalam array TFIDF untuk keperluan normalisasi TF-IDF
        self.max_tfidf = 0
        self.min_tfidf = 0

    # Kalkulasi frekuensi kemunculan tiap term didalam dokumen dengan rumus (Number of term appear in the document/total term in the document)
    def CalculateTermFrequence(self):
        for index, document in self.documents.items():
            self.tf[index] = {}
            for term in document:
                if term in self.tf[index]:
                    self.tf[index][term] += 1
                else:
                    self.tf[index][term] = 1
            self.tf[index] = {term: value/sum(self.tf[index].values()) for term, value in self.tf[index].items()}
    
    # Kalkulasi Inverse Document Frequency, yang dimana semakin kecil maka semakin sering muncul dengan rumus log(Total Documents/Number the term appear in all documents)
    def CalculateInverseDocumentFrequency(self):
        documents_length = len(documents)
        self.idf_token = set()

        for document in self.documents.values():
            self.idf_token.update(document)

        for token in self.idf_token:
            self.idf[token] = math.log(documents_length/sum(1 for doc in self.documents.values() if token in doc))
        
    # Kalkulasi TFIDF dengan rumus (TERM_ITF*Calculated_IDF_TERM)
    def CalculateTFIDF(self):
        for index, document in self.tf.items():
            self.tfidf[index] = {}
            for word, value in document.items():
                self.tfidf[index][word] = value*self.idf[word]
        
        flattened_value = [value for doc in self.tfidf.values() for value in doc.values()]
        self.max_tfidf = max(flattened_value)
        self.min_tfidf = min(flattened_value)

    def Normalize(self,value):
        return (value-self.min_tfidf)/(self.max_tfidf-self.min_tfidf)
    
    # Memanggil Semua Fungsi untuk train TFIDF yang akan digunakan di Model, dan digunakan untuk mengconvert (predict text(input)) menjadi Value TF-IDF
    def Train(self, documents = {}):
        self.documents = documents
        self.CalculateTermFrequence()
        self.CalculateInverseDocumentFrequency()
        self.CalculateTFIDF()
        
        return self.tfidf
    
    # Kalkulasi TFIDF Terhadap (predict text(input)) dengan IDF yang sudah dikalkulasi di fungsi train/IDF dari data train untuk menunjukkan keunikan setiapterm pada (predict text(input))
    def CalculatePredictTextTFIDF(self, document):
        tf = {}
        for word in document:
            if word in tf:
                tf[word]+=1
            else:
                tf[word]=1
        tf = {key: value/sum(tf.values()) for key, value in tf.items()}

        tfidf = {}
        for word, value in tf.items():
            if word in self.idf_token:
                tfidf[word] = value*self.idf[word]
        
        return tfidf

In [213]:
documents = {
    0 : ['text', 'text', 'text'],
    1 : ['text', 'macan', 'lily'],
    2 : ['macan', 'macan', 'lily'],
    3 : ['macan', 'text', 'lily'],
    4 : ['text', 'lily', 'lily'],
    5 : ['text', 'text', 'lily'],
    6 : ['text', 'macan', 'rere'],
}

In [214]:
vectorizer = TFIDFVectorizer()
vectorizer.Train(documents)
print(vectorizer.tfidf)
print(vectorizer.CalculatePredictTextTFIDF(document=['text', 'text', 'text']))

{0: {'text': 0.15415067982725836}, 1: {'text': 0.05138355994241945, 'macan': 0.18653859597847422, 'lily': 0.11215741220707096}, 2: {'macan': 0.37307719195694844, 'lily': 0.11215741220707096}, 3: {'macan': 0.18653859597847422, 'text': 0.05138355994241945, 'lily': 0.11215741220707096}, 4: {'text': 0.05138355994241945, 'lily': 0.2243148244141419}, 5: {'text': 0.1027671198848389, 'lily': 0.11215741220707096}, 6: {'text': 0.05138355994241945, 'macan': 0.18653859597847422, 'rere': 0.648636716351771}}
{'text': 0.15415067982725836}
